In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install Pillow
! pip install pytesseract
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' 
!python -m pip install torchvision tesseract
!git clone https://github.com/facebookresearch/detectron2.git
!python -m pip install -e detectron2
!pip3 install labelbox[data]
!pip install openai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.3 MB/s eta 0:00:0

In [2]:
# model train and predict libraries
from datasets import load_dataset 
from PIL import Image, ImageDraw, ImageFont
import ast
from transformers import LayoutLMv2Processor
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D
from transformers import pipeline
from torch.utils.data import DataLoader
from transformers import LayoutLMv2ForTokenClassification, AdamW
import torch
from tqdm.notebook import tqdm # 순회 가능한 객체의 진행률을 progress bar로 표현해주고 남은 시간 정보 알려줌
import pandas as pd



In [7]:
import json

text=[]
location=[]
lable=[]
for i in range(1,48):
  with open("/content/drive/MyDrive/클컴/train_data_image/train"+str(i)+".json") as json_:
    load_json = json.load(json_)
  images=load_json['images'][0]
  temp_text=[]
  temp_location=[]
  for i in load_json['images'][0]['fields']:
    temp_text.append(i['inferText'])
    temp_location.append([int(i['boundingPoly']['vertices'][0]['x']),int(i['boundingPoly']['vertices'][0]['y']),int(i['boundingPoly']['vertices'][1]['x']),int(i['boundingPoly']['vertices'][2]['y'])])


  temp_lable=[0 for _ in range(len(temp_text))]


  text.append(temp_text)
  location.append(temp_location)
  lable.append(temp_lable)


In [8]:
embedding_loaction=[]
from PIL import Image, ImageDraw, ImageFont
for i in range(1,48):
  image_path = "/content/drive/MyDrive/클컴/train_data_image/train"+str(i)+".jpeg"
  image = Image.open(image_path)
  temp_embedding_loaction=[]
  for j in range(len(location[i-1])):
    temp=[]
    temp.append(int(location[i-1][j][0]/image.size[0]*1000))
    temp.append(int(location[i-1][j][1]/image.size[1]*1000))
    temp.append(int(location[i-1][j][2]/image.size[0]*1000))
    temp.append(int(location[i-1][j][3]/image.size[1]*1000))
    temp_embedding_loaction.append(temp)
  embedding_loaction.append(temp_embedding_loaction)

In [9]:
tt=[0 for _ in range(len(text))]
image_path=["train"+str(i)+".jpeg" for i in range(1,48)]

In [10]:
import pandas as pd
df=pd.DataFrame(tt,columns=['text'])
df['text']=text
df['loc']=embedding_loaction
df['ner_tags']=lable
df['image_path']=image_path

In [11]:
df.to_csv('train_data.csv',index=False)

In [12]:
from datasets import load_dataset, filesystems, DatasetDict

data = load_dataset("csv", data_files='./train_data.csv', split="train")  # path to your file


# Split into 70% train, 30% test + validation
train_test_validation = data.train_test_split(test_size=0.2)

# Split 30% test + validation into half test, half validation
test_validation = train_test_validation["test"].train_test_split(test_size=0.5)

# Gather the splits  to have a single DatasetDict

dataset = DatasetDict(
    {
        "train": data,
        "validation": data,
        "test": data,
    }
)
dataset['train'].save_to_disk('./train_data')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8bcef0f60600fbff/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

In [17]:
dataset['train'][2]

{'text': "['주문번호', '..', '0068', '점', '포', '명', ':', '쥬씨', '죽전단국대점', '152-1', '주', '소', ':', '경기도', '용인시', '수지구', '죽전로', '사업자 번호', ':', '838-17-00489', '대표자명', ':', '최형인', '전화번호:', '031-262-2770', '카드종류', ':', '카카오뱅크', '카드번호', ':', '536510**********', '승인번호', ':', '23169488', '할부기간', ':', '일시불', '[신용카드승인]', '품', '명', '<-', '량', '금', '액', '1', '2,500', '체리', '0', '(Size)M', '0', '(당도)0%', '0', '(얼음)100%', '액', ':', '2,500원', '총', '2023-03-17', '12:43:02', '결제일시', ':', '번호', ':', '2303170068-1', '영수증', '문의', 'UNOSPAY>Tel.', '1800-4520', '<무인결제솔루션']",
 'loc': '[[100, 811, 125, 573], [135, 522, 156, 542], [125, 499, 125, 365], [187, 810, 187, 776], [194, 750, 194, 718], [200, 677, 200, 643], [205, 629, 205, 618], [203, 605, 203, 543], [205, 532, 204, 355], [216, 195, 210, 106], [210, 811, 210, 776], [221, 677, 221, 640], [227, 629, 227, 618], [224, 603, 224, 510], [226, 502, 226, 411], [225, 403, 222, 306], [222, 301, 217, 201], [230, 809, 243, 642], [250, 628, 250, 620], [245, 603, 249, 4

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'loc', 'ner_tags', 'image_path'],
        num_rows: 47
    })
    validation: Dataset({
        features: ['text', 'loc', 'ner_tags', 'image_path'],
        num_rows: 47
    })
    test: Dataset({
        features: ['text', 'loc', 'ner_tags', 'image_path'],
        num_rows: 47
    })
})

In [ ]:
# 'LayoutLMv2Processor' 을 이용하여서 이전까지 전처리 했던 데이터를 이용하여 모델 학습에 맞는 형태로 전처리를 한번 더 해준다.

from datasets import load_metric, load_from_disk,load_dataset,Features, Sequence, ClassLabel, Value, Array2D, Array3D


#labels=> 예측값의 layout 명칭
labels = ['others','item']
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}

#huggingface : https://huggingface.co/microsoft/layoutlmv2-base-uncased 의 pretrained 된 모델을 사용
#입력된 데이터를 모델에 학습을 시킬수 있는 데이터로 전환하는 전처리 하기 위한 모델이다.

processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")


# 전처리 데이터의 데이터 타입을 지정 
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
})



def preprocess_data(example):
    processor.feature_extractor.apply_ocr=False
    images=[Image.open("/content/drive/MyDrive/클컴/train_data_image/"+str(i)).convert('RGB') for i in example['image_path']]
    words = [ast.literal_eval(i) for i in example['text']]
    boxes = [ast.literal_eval(i) for i in example['loc']]
    word_labels = [ast.literal_eval(i) for i in example['ner_tags']]
    print(len(word_labels[0]))
    encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                                 padding="max_length", truncation=True)
    

    return encoded_inputs

dataset = load_from_disk('./train_data')

train_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset.column_names,features=features)
train_dataset.set_format(type = 'torch')




Map:   0%|          | 0/47 [00:00<?, ? examples/s]

67


In [ ]:
test_set=train_dataset[0]
label=test_set.pop('labels').squeeze().tolist()


In [ ]:
test_set

In [ ]:
train_dataset[0]

model = LayoutLMv2ForTokenClassification.from_pretrained('microsoft/layoutlmv2-base-uncased')
output=model(**test_set)

output


In [ ]:
#모델 훈련 시작


print('train batch size',len(train_dataset))
train_dataloader = DataLoader(train_dataset,batch_size=len(train_dataset))


model = LayoutLMv2ForTokenClassification.from_pretrained('microsoft/layoutlmv2-base-uncased',
                                                          num_labels=1)

optimizer = AdamW(model.parameters(), lr= 0.00005) #

global_step = 0
num_train_epochs = 20


model.train() 
print(train_dataloader)
for epoch in range(num_train_epochs):  
    print("Epoch:", epoch)
    for batch in tqdm(train_dataloader):
      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = model(**batch) 
      loss = outputs.loss
      print(f"Loss after {global_step} steps: {loss.item()}")

      loss.backward()
      optimizer.step()
      global_step += 1  


train batch size 47


Some weights of the model checkpoint at microsoft/layoutlmv2-base-uncased were not used when initializing LayoutLMv2ForTokenClassification: ['layoutlmv2.visual.backbone.bottom_up.res4.5.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.6.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.2.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.2.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.19.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.21.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.1.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.3.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.2.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.8.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.14.conv3.norm.num_batches_tracked'

Epoch: 0


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

IndexError: ignored